In [1]:
import os,os.path as osp
import io,gzip
import functools as ft,itertools as it
import pycurl
import numpy as np,pandas as pd

In [2]:
parent_dir = osp.abspath(osp.join(os.getcwd(),os.pardir))
cfg = osp.join(parent_dir,'genelinks.cfg')
osp.exists(cfg)

True

In [3]:
chrmsm = ['1','2','3','4','5','6','7','8','9','10','11','12','13',
          '14','15','16','17','18','19','20','21','22','X','Y','MT']
dir_p1 = 'chrom-{:02d}'
dir_p2 = 'chrom-{:s}'
for x in chrmsm:
    if x.isdecimal():
        dstr = dir_p1.format(int(x))
    else:
        dstr = dir_p2.format(x)
    dirs = osp.join(parent_dir,'data',dstr)
    if not osp.exists(dirs):
        os.makedirs(dirs)
        print("created dir for %s"%x)
    else:
        print("found dir for %s"%x)

found dir for 1
found dir for 2
found dir for 3
found dir for 4
found dir for 5
found dir for 6
found dir for 7
found dir for 8
found dir for 9
found dir for 10
found dir for 11
found dir for 12
found dir for 13
found dir for 14
found dir for 15
found dir for 16
found dir for 17
found dir for 18
found dir for 19
found dir for 20
found dir for 21
found dir for 22
found dir for X
found dir for Y
found dir for MT


In [4]:
D = {}
with open(cfg,'r') as f:
    lines = [l.strip() for l in f.readlines() if l]
fn = lambda X: X.split('.')[-2]
for k,g in it.groupby(sorted(lines,key=fn),key=fn):
    D[k] = list(g)
D.keys()

dict_keys(['23andme', '23andme-exome-vcf', 'IYG', 'ancestry', 'decodeme', 'ftdna-illumina', 'genes-for-good'])

In [5]:
txt = ['23andme','23andme-exome-vcf','ancestry','genes-for-good']
gzd = ['IYG','decodeme','ftdna-illumina']

In [6]:
def read_23andme(pagebytes):
    cols = ['rsid','chrom','pos','alleles']
    sep = '\t'
    pg = io.StringIO(pagebytes)
    args = {
        'comment':'#','sep':sep,'dtype':object,'names':cols
    }
    return pd.read_csv(pg,**args)
def read_ancestry(pagebytes):
    cols = ['rsid','chrom','pos','alleles']
    sep = ','
    args = {
        'skiprows':1,'sep':sep,'dtype':object,'names':cols
    }
    filt = lambda Y: not Y.startswith('#')
    decd = lambda X: X.replace('\t',',',3).replace('\t','')
    junk = pagebytes
    fpg = filter(filt,junk.splitlines())
    junk = [decd(x) for x in fpg]
    junk = io.StringIO('\n'.join(junk))
    return pd.read_csv(junk,**args)

In [7]:
def link_grab(Links):

    reqs = []
    m = pycurl.CurlMulti()
    for url in Links:
        buff = io.BytesIO()
        handle = pycurl.Curl()
        handle.setopt(pycurl.URL, url)
        handle.setopt(pycurl.WRITEDATA, buff)
        req = (url, buff, handle)
        m.add_handle(req[2])
        reqs.append(req)
    # Perform multi-request, modified from:
    # https://github.com/rushtongarth/XnaPyCurl/blob/master/src/getters/multi.py
    num_processed=0
    num_urls = len(Links)
    while num_processed < num_urls:
        while 1:
            ret, num_handles = m.perform()
            if ret != pycurl.E_CALL_MULTI_PERFORM: break
            else: print(num_handles)
        while 1:
            num_q, ok_list, err_list = m.info_read()
            for c in ok_list:
                m.remove_handle(c)
                print("Success:",c.getinfo(pycurl.EFFECTIVE_URL))
            for c, errno, errmsg in err_list:
                m.remove_handle(c)
                print("Failed: ", c, errno, errmsg)
            num_processed = num_processed + len(ok_list) + len(err_list)
            if num_q == 0:  break
        m.select(1.0)
    return reqs


In [8]:
def file_proc(to_proc,mode='23andme',verbose=False):
    f1 = lambda Y: Y and not Y.startswith('#')
    f2 = lambda X: '-'.join(X.split('/')[-1].split('.')[::2])
    rows = {}
    if mode=='23andme':
        to_df = read_23andme
    elif mode=='ancestry':
        to_df = read_ancestry

    for u,req,h in to_proc:
        try:
            to_dec = req.getvalue().decode('ascii')
            genes = to_df(to_dec)
            rows[f2(u)] = genes
        except UnicodeDecodeError as e:
            print('Skipping %s'%u)
    return rows

In [9]:
subset = len(D['23andme'])//50
print(subset)

tograb = np.random.choice(D['23andme'],subset)
genes = link_grab(tograb)

69
Success: https://www.opensnp.org/data/519.23andme.236
Success: https://www.opensnp.org/data/607.23andme.2873
Success: https://www.opensnp.org/data/2848.23andme.1807
Success: https://www.opensnp.org/data/3313.23andme.2153
Success: https://www.opensnp.org/data/3121.23andme.2014
Success: https://www.opensnp.org/data/4038.23andme.2727
Success: https://www.opensnp.org/data/4787.23andme.3372
Success: https://www.opensnp.org/data/4509.23andme.3106
Success: https://www.opensnp.org/data/5185.23andme.3866
Success: https://www.opensnp.org/data/2578.23andme.1597
Success: https://www.opensnp.org/data/2157.23andme.1309
Success: https://www.opensnp.org/data/4402.23andme.3002
Success: https://www.opensnp.org/data/7726.23andme.6073
Success: https://www.opensnp.org/data/3563.23andme.2747
Success: https://www.opensnp.org/data/3388.23andme.2203
Success: https://www.opensnp.org/data/107.23andme.51
Success: https://www.opensnp.org/data/5936.23andme.4400
Success: https://www.opensnp.org/data/4493.23andme.

In [10]:
L = file_proc(genes)

Skipping https://www.opensnp.org/data/4195.23andme.2848
Skipping https://www.opensnp.org/data/607.23andme.2873
Skipping https://www.opensnp.org/data/6832.23andme.5214


In [23]:
dir_p1 = 'chrom-{:02d}'
dir_p2 = 'chrom-{:s}'
fnamed = '{:s}.frame.gz'
for k,v in L.items():
  for g,rws in v.groupby('chrom'):
    if g.isdecimal():
      f = osp.join(parent_dir,'data',dir_p1,fnamed).format(int(g),k)
    else:
      f = osp.join(parent_dir,'data',dir_p2,fnamed).format(g,k)
    rws.to_pickle(f,compression='gzip')
  print("compressed: %s"%k)

compressed: 584-269
compressed: 4056-2741
compressed: 980-486
compressed: 2397-1489
compressed: 2135-1301
compressed: 4907-3461
compressed: 1515-833
compressed: 2825-1786
compressed: 4038-2727
compressed: 7381-5733
compressed: 3455-2251
compressed: 7715-6058
compressed: 2507-1537
compressed: 6561-4958
compressed: 3787-5497
compressed: 7089-5488
compressed: 2189-1337
compressed: 4482-3079
compressed: 5945-4438
compressed: 1325-706
compressed: 7255-5633
compressed: 4030-5110
compressed: 519-236
compressed: 4014-2705
compressed: 3037-3696
compressed: 2038-1219
compressed: 4787-3372
compressed: 4509-3106
compressed: 5225-3738
compressed: 2157-1309
compressed: 2222-1369
compressed: 2578-1597
compressed: 5185-3866
compressed: 1980-1166
compressed: 3172-2057
compressed: 7251-5630
compressed: 3388-2203
compressed: 7726-6073
compressed: 4402-3002
compressed: 1500-819
compressed: 437-190
compressed: 6827-5210
compressed: 3313-2153
compressed: 3121-2014
compressed: 2848-1807
compressed: 3563-2747

In [24]:
del genes
del L

In [26]:
u = set()
for k,v in L.items():
    rows = set(v.rsid.unique())
    diff = rows - u
    if len(diff)>0:
        u |= rows

In [27]:
len(u)

1552626

In [30]:
del L

In [28]:
import pickle

In [29]:
with open('../data/sliced.pkl','wb') as f:
    pickle.dump(L,f)

MemoryError: 

In [44]:
k1,k2 = list(L.keys())[:2]
f1,f2 = L[k1],L[k2]
f1.chrom.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X',
       'Y', 'MT'], dtype=object)